In [2]:
import pandas as pd
import json
import os
from datetime import datetime
import chart_studio.plotly as py
import cufflinks as cf
cf.go_offline(connected=True)

# 회복자수 추이 / 사망자수 추이
# 일단 회복자수만 보여주기
PATH = "00_Material(Uploaded)/COVID-19-master/csse_covid_19_data/csse_covid_19_daily_reports/"

with open("00_Material(Uploaded)/COVID-19-master/csse_covid_19_data/country_convert.json", 'r', encoding='utf-8-sig') as json_file:
    json_data = json.load(json_file)
    # print(json_data)

def country_name_convert(row):
    if row['Country_Region'] in json_data:
        return json_data[row['Country_Region']]
    return row['Country_Region']

def create_dateframe(filename):
    doc = pd.read_csv(PATH + filename, encoding="utf-8-sig")
    try:
        doc = doc[['Country_Region', 'Recovered' ]]  # 1. 특정 컬럼만 선택해서 데이터프레임 만들기
    except:
        doc = doc[['Country/Region', 'Recovered' ]]  # 1. 특정 컬럼만 선택해서 데이터프레임 만들기
        doc.columns = ['Country_Region', 'Recovered' ]
    doc = doc.dropna(subset=['Recovered'])
    doc['Country_Region'] = doc.apply(country_name_convert, axis=1)
    doc = doc.astype({'Recovered' : 'int64'})
    doc = doc.groupby('Country_Region').sum()

    date_col = filename.split('.')[0].lstrip("0").replace('-', '/')
    doc.columns = [date_col + "_Recovered"]
    return doc


country_info = pd.read_csv("00_Material(Uploaded)/COVID-19-master/csse_covid_19_data/UID_ISO_FIPS_LookUp_Table.csv")

def generate_dateframe_by_path(PATH):
    file_list, csv_list = os.listdir(PATH), list()
    first_doc = True

    for file in file_list:
        if file.split('.')[-1] == 'csv':
            csv_list.append(file)

    csv_list.sort(key = lambda x: datetime.strptime(x, '%m-%d-%Y.csv'))
    
    for file in csv_list:
        doc = create_dateframe(file)
        if first_doc:
            final_doc, first_doc = doc, False
        else:
            final_doc = pd.merge(final_doc, doc, how='outer', left_index=True, right_index=True)

    final_doc = final_doc.fillna(0)
    return final_doc        

doc = generate_dateframe_by_path(PATH)
doc.iplot(kind='scatter', mode='lines+markers')

# doc.head()

ValueError: 
    Invalid value of type 'builtins.str' received for the 'mode' property of scatter
        Received value: 'Lines+markers'

    The 'mode' property is a flaglist and may be specified
    as a string containing:
      - Any combination of ['lines', 'markers', 'text'] joined with '+' characters
        (e.g. 'lines+markers')
        OR exactly one of ['none'] (e.g. 'none')